In [ ]:
# Imports
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# Define date to conduct the analysis with (default set to today, override if required)
date=datetime.today().strftime('%Y.%m.%d')
#date='2025.03.21'
print('Data preparation started for date: '+date)

# Read the data from the CSV files
complianceData = pd.read_csv('compliance_'+date+'.csv')
vendorData = pd.read_csv('vendors_'+date+'.csv')
productData = pd.read_csv('products_'+date+'.csv')
modelVersions = pd.read_csv('modelVersions_'+date+'.csv')

# device Types from Matter-1.4-Device-Library-Specification.pdf
device_Types = pd.read_csv('device_types.csv')

In [ ]:
#Analyse creators of entries
print('Vendor creatores: '+str(len(pd.unique(vendorData['creator']))))
print('Compliance creators: '+str(len(pd.unique(complianceData['owner']))))
print('Product creatores: '+str(len(pd.unique(productData['creator']))))
print('ModelVersions creatores: '+str(len(pd.unique(modelVersions['creator']))))

In [ ]:
# Analyse Vendors
print(len(vendorData.index))
percent_missing = vendorData.isnull().sum() * 100 / len(vendorData)
missing_value_df = pd.DataFrame({'percent of missing values': round(percent_missing)})
print(missing_value_df)

vendors_without_certification = pd.merge(vendorData, complianceData, left_on='vendorID', right_on='vid', how='left')
vendors_without_certification=vendors_without_certification[vendors_without_certification['pid'].isnull()]
print('Vendors without certification: '+str(len(vendors_without_certification)))

vendors_without_model = pd.merge(vendorData, productData, left_on='vendorID', right_on='vid', how='left')
vendors_without_model=vendors_without_model[vendors_without_model['pid'].isnull()]
print('Vendors without model: '+str(len(vendors_without_model)))


In [ ]:
# Analyse DeviceModel
print(len(productData.index))
productData['nameMatchesLabel'] = np.where((productData['productName'] == productData['productLabel']) , productData['productName'], np.nan)

percent_missing = productData.isnull().sum() * 100 / len(productData)
missing_value_df = pd.DataFrame({'percent of missing values': round(percent_missing)})
print(missing_value_df)

In [ ]:
# Analyse DeviceModelVersion
print(len(modelVersions.index))
percent_missing = modelVersions.isnull().sum() * 100 / len(modelVersions)
missing_value_df = pd.DataFrame({'percent of missing values': round(percent_missing)})
print(missing_value_df)

In [ ]:
# Analyse Compliance
print(len(complianceData.index))
percent_missing = complianceData.isnull().sum() * 100 / len(complianceData)
missing_value_df = pd.DataFrame({'percent of missing values': round(percent_missing)})
print(missing_value_df)

# There might be multiple certified versions of a single device model. The next lines identify how many unique device models are certified.
uniqueVendorIdAndProductId=pd.DataFrame()
uniqueVendorIdAndProductId['vidPid']=complianceData['vid'].astype(str) + complianceData['pid'].astype(str)
print('Unique certified vendor and product ids: '+str(len(uniqueVendorIdAndProductId['vidPid'].unique())))

In [ ]:
#reduce Datasets
vendorData = vendorData.drop(columns=['companyPreferredName', 'creator'])
complianceData = complianceData.drop(columns=['reason',	'owner', 'history', 'cDCertificateId', 'certificationRoute', 'programType', 'programTypeVersion', 'compliantPlatformUsed', 'compliantPlatformVersion', 'transport', 'familyId', 'supportedClusters', 'OSVersion', 'parentChild', 'certificationIdOfSoftwareComponent'])
productData = productData.drop(columns=['commissioningModeInitialStepsInstruction', 'commissioningModeSecondaryStepsInstruction', 'lsfRevision', 'creator'])
modelVersions = modelVersions.drop(columns=['cdVersionNumber', 'softwareVersionValid', 'otaFileSize', 'otaChecksum', 'otaChecksumType', 'creator'])


In [ ]:
# Merge the data
merged_df = pd.merge(productData, vendorData, left_on='vid', right_on='vendorID', how='left')
# Create column Device Type ID Int as helper column for the datamerge
device_Types['Device Type ID Int'] = device_Types['Device Type ID'].apply(lambda hex: int(hex,0))
merged_df = pd.merge(merged_df, device_Types,  left_on='deviceTypeId', right_on='Device Type ID Int', how='left')
#remove helper column
merged_df = merged_df.drop(columns=['Device Type ID Int'])

merged_compliance_df = pd.merge(complianceData, merged_df, left_on=['pid', 'vid'], right_on=['pid', 'vid'], how='left') 

# remove rows without matching device model
merged_compliance_df_without_device_model = merged_compliance_df[merged_compliance_df['productName'].isnull()]
merged_compliance_df = merged_compliance_df.drop(merged_compliance_df[merged_compliance_df['productName'].isnull()].index)
print('Certification without matching device models removed from dataset: '+ str(len(merged_compliance_df_without_device_model)))

# merge software model version data to the dataframe
merged_compliance_version_df = pd.merge(merged_compliance_df, modelVersions, left_on=['pid', 'vid', 'softwareVersion'], right_on=['pid', 'vid', 'softwareVersion'], how='left') 

#prepare the secound dataframe (uncertified products)
merged_df['vidPid']=merged_df['vid'].astype(str) + merged_df['pid'].astype(str)
complianceData['vidPid']=complianceData['vid'].astype(str) + complianceData['pid'].astype(str)
merged_df_uncertified = merged_df[~merged_df['vidPid'].isin(complianceData['vidPid'])]
#merged_df_uncertified = merged_df_uncertified.drop(columns=['vidPid'])
merged_df_uncertified = pd.merge(merged_df_uncertified, modelVersions, left_on=['pid', 'vid'], right_on=['pid', 'vid'], how='left') 

# Save Merged Data to CSV file
merged_df_uncertified.to_csv('uncertified_devices_'+date+'.csv', index=False)
merged_compliance_version_df.to_csv('certified_devices_'+date+'.csv', index=False)